# Automatic differentiation with Zygote
- What is automatic differentiation (AD)
- How to use chain rule to compute compilcated gradients
- How Zygote implements this using so called adjoints
- How to design your own adjoint
    + How to check if it is correctly computed
    + Some advanced examples

In [1]:
using Zygote

## Simple example
Let's start with a simple scalar function of one variable $f(x) = 5x + 3$.

In [26]:
f(x) = 5x + 3;

The exact result of derivative with respect to `x` is of course simple constant function $\frac{df}{dx} = 5$.

In [27]:
df(x) = 5;

Giving us the expected results

In [28]:
f(10), df(10)

(53, 5)

Our goal is to compute *exact* derivatives of scalar functions (mostly) in an automatic way.

In this simple case Zygote alows us to use the adjoint/transpose operator `'` to compute the same derivative but automatically.

In [29]:
f(10), f'(10)

(53, 5)

The important thing is that Zygote does **source to source** AD, where the derivatives itself are automatically generated functions and are treated by the compiler the same way as if you were to write them yourselves.

In [30]:
@which f'(10)

(::Zygote.var"#39#40")(x) in Zygote at /home/honza/.julia/packages/Zygote/1GXzF/src/compiler/interface.jl:57

In [31]:
@code_typed f'(10)

CodeInfo(
1 ─      goto #3
2 ─      $(Expr(:meta, :inline))
3 ┄      goto #4
4 ─      goto #5
5 ─ %5 = Base.mul_int(5, 1)::Int64
└──      goto #7
6 ─      $(Expr(:meta, :inline))
7 ┄      goto #8
8 ─      goto #9
9 ─      return %5
) => Int64

Looks a little bit more complicated, but when we print the code in machine form we see that this all boils down to a function returning `5`.

In [32]:
@code_native f'(10)

	.text
; ┌ @ interface.jl:57 within `#39'
	movl	$5, %eax
	retq
	nopw	%cs:(%rax,%rax)
; └


Though the notation of `'` operator overloading is neat, it works only for the simplest scalar functions of single variable.

In [33]:
g(x,y) = 2x + 3y

g (generic function with 1 method)

In [34]:
g'(1,1)

MethodError: MethodError: no method matching (::Zygote.var"#39#40"{typeof(g)})(::Int64, ::Int64)
Closest candidates are:
  #39(::Any) at /home/honza/.julia/packages/Zygote/1GXzF/src/compiler/interface.jl:57

Once we move to functions of more variable the thing we want to compute is `gradient`, which returns the derivative with respect to each of the input arguments.

In [35]:
gradient(g, 1, 1)

(2, 3)

In [36]:
@code_typed gradient(g, 1, 1)

CodeInfo(
1 ─      goto #3
2 ─      $(Expr(:meta, :inline))
3 ┄      goto #4
4 ─      goto #5
5 ─ %5 = Base.mul_int(3, 1)::Int64
│   %6 = Base.mul_int(2, 1)::Int64
└──      goto #7
6 ─      $(Expr(:meta, :inline))
7 ┄ %9 = Core.tuple(%6, %5)::Tuple{Int64,Int64}
└──      goto #8
8 ─      return %9
) => Tuple{Int64,Int64}

Which also works on the previous example with function `f`, but instead of returning single number it gives a tuple.

In [37]:
gradient(f, 10)

(5,)

Let's compute the backpropagation for a single layer neural network
$$
n(x) = Wx + b
$$

In [38]:
W, b = rand(2, 3), rand(2);
net(x) = W*x .+ b;
agg(n) = sum(n); # simple aggregation

In [40]:
agg(net([1,3,2]))

6.8233560637319375

By default the gradient is computed with respect to the input variables.

In [41]:
gradient((x) -> agg(net(x)), [1,2,3])

([0.9573036701702753, 1.0742578227878743, 0.9663246784951718],)

If you write it down this should correspont to taking the sum of rows of the weight matrix `W`.

In [42]:
sum(W,dims=1)

1×3 Array{Float64,2}:
 0.957304  1.07426  0.966325

In the field of machine learning we are however usually interested in taking derivatives with respect to the weights and biases. For that Zygote uses the concept of `Params` parameters, which are in turn used in the Flux library.

In [43]:
p = Params([W, b])

Params([[0.12079695301991178 0.6115448541502737 0.09264696995665744; 0.8365067171503635 0.4627129686376006 0.8736777085385143], [0.24506599042207244, 0.4655635777856233]])

Running the gradient instead, returns a dictionary with entries for each paramter in `p`.

In [44]:
grads = gradient(() -> agg(net([1,2,3])), p)

Grads(...)

Gradient dictionary can be indexed directly with the parameter we want the gradient to.

In [45]:
grads[W]

2×3 Array{Float64,2}:
 1.0  2.0  3.0
 1.0  2.0  3.0

Notice that the gradient is a matrix, even though one would expect it to be vector as you were though in calculus. This representation of gradient is more convenient as the update step of gradient descent algortihm looks really simple.

In [46]:
α = 0.01
W .-= α.*grads[W]

2×3 Array{Float64,2}:
 0.110797  0.591545  0.062647
 0.826507  0.442713  0.843678

But keep in mind that from calculus point of view we are still working with scalar functions of multiple variables 
$$
W = 
\begin{pmatrix}
W_{11} & W_{12} & W_{13}\\
W_{21} & W_{22} & W_{23}
\end{pmatrix} \\
b = 
\begin{pmatrix}
b_{1} \\
b_{2}
\end{pmatrix}
$$
and gradients should be in a vector form

In [47]:
vec(grads[W])

6-element Array{Float64,1}:
 1.0
 1.0
 2.0
 2.0
 3.0
 3.0

In [48]:
vec(grads[b])

2-element FillArrays.Fill{Float64,1,Tuple{Base.OneTo{Int64}}}:
 1.0
 1.0

## Under the hood (forward/backward)

Let's look now at how this automatic differentiation works internally. Let's define more complicated nested function such as
$$
s(x) = 5sin(x^2)
$$

In [49]:
s(x) = sin(x^2) * 5

s (generic function with 1 method)

I would like to point out that there is really no much difference from how Zygote computes the derivatives and how you may go about it. What we do is applying the following basic rules to compute the result.
$$
\begin{align}
\frac{d}{dx} x &= 1 \\
\frac{d}{dx} (-u) &= - \frac{du}{dx} \\
\frac{d}{dx} (u + v) &= \frac{du}{dx} + \frac{dv}{dx} \\
\frac{d}{dx} (uv) &= v \frac{du}{dx} + u \frac{dv}{dx} \\
\frac{d}{dx} (u / v) &= (v \frac{du}{dx} - u \frac{dv}{dx}) / v^2 \\
\frac{d}{dx} u^n &= n u^{n-1} \\
\frac{d}{dx} u(v) &= \frac{du}{dv} \frac{dv}{dx} \\
\vdots
\end{align}
$$

The most important thing to understand is that due to Julia's superior meta language capabilities, it is able to understand the code of a function `s` and decompose it in order to apply these rules by pattern matching. Result of which is so called Wengert list of `s`
```julia
y1 = x ^ 2
y2 = sin(y1)
y3 = y2 * 5
```
In other words, we have been able to produce a list of assigments, each of which we are able to differentiate using the standard calculus toolbox. In order to compute the derivative itself we just have to chain the elementary derivatives $\frac{dy_1}{dx}, \frac{dy_2}{dy_1}, \frac{ds}{dy_2}$ and voila 
$$
\frac{ds}{dx} = \frac{dy_1}{dx} \times \frac{dy_2}{dy_1} \times \frac{ds}{dy_2}
$$

Due to the fact that multiplication is asociative, there are two ways how to compute the derivate. 
We can go either from either evaluating first $\frac{dy_1}{dx} \times \frac{dy_2}{dy_1}$ first, or $\frac{dy_2}{dy_1} \times \frac{ds}{dy_2}$?

It's easier to see the distinction if we think algorithmically. Given some
enormous Wengert list with $n$ instructions, we have two ways to differentiate
it:

### Forward
Start with the known quantity $\frac{dy_0}{dx} = \frac{dx}{dx} = 1$
at the beginning of the list. Look up the derivative for the next instruction
$\frac{dy_{i+1}}{dy_i}$ and multiply out the top, getting $\frac{dy_1}{dx}$,
$\frac{dy_2}{dx}$, ... $\frac{dy_{n-1}}{dx}$, $\frac{dy}{dx}$. Because we
walked forward over the Wengert list, this is called *forward mode*. Each
intermediate derivative $\frac{dy_i}{dx}$ is known as a *perturbation*.

### Reverse
Start with the known quantity $\frac{dy}{dy_n} = \frac{dy}{dy} = 1$
at the end of the list. Look up the derivative for the previous instruction
$\frac{dy_i}{dy_{i-1}}$ and multiply out the bottom, getting
$\frac{dy}{dy_n}$, $\frac{dy}{dy_{n-1}}$, ... $\frac{dy}{dy_1}$,
$\frac{dy}{dx}$. Because we walked in reverse over the Wengert list, this is
called *reverse mode*. Each intermediate derivative $\frac{dy}{dy_i}$ is known
as a *sensitivity*.


Zygote uses the latter method as there are huge performance benefits, when computing derivatives with respect to milions of parameters as is the case with ML models.
- taking derivatives of scalar function with respect to more than one variable requires only one pass
- $\frac{dy_i}{dx}$ is a huge Jacobi matrix (`length(y_i) * length(x)`), whereas $\frac{dy}{dy_i}$ is represented more compactly (`length(y) * length(y_i)`)

## How is it implemented?
Even though the way the Wengert lists are created is really important here we will focus only on the differentiation itself.

Going back to our function `s` and suppose that Zygote has already parsed the function into a Wengert list.

In [50]:
w1(x) = x ^ 2
w2(x) = sin(x)
w3(x) = x * 5.0

function s(x)
    y1 = w1(x)
    y2 = w2(y1)
    y3 = w3(y2)
end

s (generic function with 1 method)

Just as a reference, here is the result and gradient that we want to compute.

In [51]:
s(4.0), gradient(s, 4.0)[1]

(-1.4395165833253265, -38.30637921293538)

In Zygote `gradient` is just a wrapper for internal function called `pullback` (or more precisely even more internal `_pullback` and `Pullback` type), which returns both the value of a given function and something called **adjoint** or **adjoint function** representing the gradient. 

In [52]:
s_val, s_back = pullback(s, 4.0)

(-1.4395165833253265, Zygote.var"#37#38"{typeof(∂(s))}(∂(s)))

What should we pass that adjoint function to get our gradient? For that we need to understand what the adjoint represents.

Looking back to the differentiated Wengert list
$$
\frac{ds}{dx} = \frac{dy_1}{dx} \times \frac{dy_2}{dy_1} \times \frac{dy_3}{dy_2},
$$
we can pull back the adjoint for every other intermediate result in the list, while feeding the interemediate value to the next function in the list as the derivative for `y2` is computed not at `x = 4.0` but at `y1`.

In [53]:
y1_val, y1_back = pullback(w1, 4.0)
y2_val, y2_back = pullback(w2, y1_val)
y3_val, y3_back = pullback(w3, y2_val)

(-1.4395165833253265, Zygote.var"#37#38"{typeof(∂(w3))}(∂(w3)))

As expected `y3_val == s_val`. In this way we have thus evaluated the function `s` itself, however we got also the adjoint functions for each intermediate result of that Wengert list, which can be composed in **reverse** order to compute the derivative of the whole list.

In [54]:
y1_back(y2_back(y3_back(1.0)[1])[1])[1]

-38.30637921293538

And this is the derivative we were looking for, where the input `1.0` corresponds to the fact that $\frac{dy_3}{dy_3} = 1$ and what the adjoint function is really computing is the multiplication of two subsequent derivatives/gradients in the chain, i.e.

In [55]:
y3_back(1.0)

(5.0,)

computes $\frac{dy_3}{dy_2} \times \frac{dy_3}{dy_3} = \frac{ds}{dy_2}$, this result is plug again into the pullback of `y2` to compute $\frac{dy_2}{dy_1} \times \frac{dy_3}{dy_2} = \frac{ds}{dy_1}$ and so on until the derivative is computed.

In [56]:
y2_back(y3_back(1.0)[1])

(-4.788297401616923,)

In [57]:
y1_back(y2_back(y3_back(1.0)[1])[1])[1]

-38.30637921293538

The adjoint functions are thus not derivatives/gradients themselves, but they implemented as the multiplication of two derivatives/gradients in the chain. In other words, given a derivative with respect to it's output, they return derivative with respect to it's inputs.

In case of scalar functions of one variable, this is all there is to know about how the adjoint works, however things get a little messy, when we start working with vector functions of multiple variables. such as the following example of composition of functions.

$$
\begin{align}
f(x) &= 
\begin{pmatrix}
{x_{1} + x_{2}} \\
{x_{1}x_{2}}
\end{pmatrix} \\
g(x) &= 
\begin{pmatrix}
e^{(x_{1} + x_{2})} \\
{\ln(x_{1}) + \ln(x_{2})}
\end{pmatrix} \\
h(x) &= g\left(f(x)\right)
\end{align}
$$
For comparison I have precomputed the Jacobi matrices for each of the function.

In [58]:
f(x) = [x[1] + x[2], x[1]*x[2]]
df(x) = [1.0 1.0; x[2] x[1]];

g(y) = [exp(y[1] + y[2]), log(y[1])+log(y[2])]
dg(y) = [exp(y[1] + y[2]) exp(y[1] + y[2]); 1/y[1] 1/y[2]];

In [59]:
h(x) = g(f(x))

h (generic function with 1 method)

For the derivative of $h$ we use the chain rule
$$
h'(x) = g'\left(f(x)\right)f'(x),
$$
where $g'$ and $f'$ are the corresponding Jacobi matrices.

In [60]:
dh(x) = dg(f(x))*df(x)

dh (generic function with 1 method)

What we expect to compute in this case when we ask Zygote to compute pullback of vector functions?

In [61]:
x_val = [1.0, 2.0]
f_val, f_back = pullback(f, x_val)

([3.0, 2.0], Zygote.var"#37#38"{typeof(∂(f))}(∂(f)))

In [63]:
df(x_val)

2×2 Array{Float64,2}:
 1.0  1.0
 2.0  1.0

As we are working with vector functions of multiple variables, we the input for adjoint corresponds to either if we are computing the derivative of $f_1$ 
$$
\nabla_x f_1 =  
\begin{pmatrix}
\frac{\partial f_1}{\partial x_1} &
\frac{\partial f_1}{\partial x_2}
\end{pmatrix}
$$

In [64]:
f_back([1.0, 0.0])

([1.0, 1.0],)

or $f_2$
$$
\nabla_x f_2 =  
\begin{pmatrix}
\frac{\partial f_2}{\partial x_1} &
\frac{\partial f_2}{\partial x_2}
\end{pmatrix}
$$

In [65]:
f_back([0.0, 1.0])

([2.0, 1.0],)

The adjoint function now corresponds to left multiplication the Jacobi matrix with the gradient to it's output.
$$
\nabla \cdot f'
$$
( in the code of adjoints there is usually additional transposition of ∇ to a row vector)

In order to obtain the whole Jacobi matrix of a vector function we have to evaluate the reverse step multiple times, however this is usually not necessary in ML applications. Now back to the function `h`, whose Wenger list representation would look like this:

In [66]:
function h(x)
    y1 = f(x)
    y2 = g(y1)
end

h (generic function with 1 method)

In [67]:
y1_val, y1_back = pullback(f, x_val)
y2_val, y2_back = pullback(g, y1_val)

([148.4131591025766, 1.791759469228055], Zygote.var"#37#38"{typeof(∂(g))}(∂(g)))

This should correspond to running

In [68]:
h_val, h_back = pullback(h, x_val)

([148.4131591025766, 1.791759469228055], Zygote.var"#37#38"{typeof(∂(h))}(∂(h)))

Choosing to compute $\nabla_x h_2$, we get three time the same result

In [69]:
h_back([0.0, 1.0])

([1.3333333333333333, 0.8333333333333333],)

In [70]:
y1_back(y2_back([0.0, 1.0])[1])

([1.3333333333333333, 0.8333333333333333],)

In [71]:
dh(x_val)[2,:]

2-element Array{Float64,1}:
 1.3333333333333333
 0.8333333333333333

## Footnote: Matrix valued functions
Now that we know how Zygote works on vector functions, let's complicate the things a little bit for ourselves with matrix-matrix valued functions such as the linear layer of neural network from the beggining. 

In [72]:
W, b = rand(2, 3), rand(2);
n(X) = W*X .+ b

n (generic function with 1 method)

`X` is now a matrix, representing a batch of samples.

In [73]:
X_val = rand(3, 10)

3×10 Array{Float64,2}:
 0.0947169  0.474544   0.585249  0.336051  …  0.0952857  0.851439  0.946595
 0.770825   0.402985   0.311854  0.84953      0.252944   0.404033  0.195099
 0.277271   0.0824026  0.830371  0.03958      0.582057   0.799171  0.37028

Even though that is usually not the case, let's try to compute the derivative of $n$ with respect to the matrix of values $X$.

In [74]:
n_val, n_back = pullback(n, X_val)

([0.9208945484196884 0.6115449418793542 … 1.2082981759037437 0.7825413548852491; 1.4405468289608332 1.5720248740634677 … 2.440460272135355 2.174149170484616], Zygote.var"#37#38"{typeof(∂(n))}(∂(n)))

As expected the output another matrix of size 2x10

In [75]:
n_val

2×10 Array{Float64,2}:
 0.920895  0.611545  1.0721   0.900524  …  0.706549  1.2083   0.782541
 1.44055   1.57202   2.18677  1.52039      1.52973   2.44046  2.17415

Now imagine function `n` to be a part of big model with some scalar loss function, whose derivative we compute in reverse mode. When we get to it we have already collected some gradient with respect to the output of `n`, which has the same size as `n_val` (rememmber the `vec` transformation of matrices to vector is just a matter of conveniece), therefore the input for our adjoint function `n_back` will have the same dimensions.

In [76]:
Δ = rand(size(n_val)...)

2×10 Array{Float64,2}:
 0.0246601  0.686651  0.24968   0.357603  …  0.656952  0.823438  0.017808
 0.829612   0.334416  0.480006  0.745095     0.523166  0.557017  0.749535

Giving us the same dimensions as the input of the matrix function.

In [77]:
n_back(Δ)[1]

3×10 Array{Float64,2}:
 0.796395  0.531047  0.533848  0.819372  …  0.701281  0.785133  0.718136
 0.225205  0.637207  0.320391  0.473134     0.659934  0.802744  0.199856
 0.606593  0.696687  0.508268  0.768943     0.811103  0.946425  0.545054

In calculus however we are usually not taught how to differentiate such functions, as they are just hidden vector functions of multiple variables, whose Jacobi matrices are computed in the same way, however making sense of it is may not be as straightforward for everyone. 

Take for example the matrix-matrix multiplication in the function `n`. Suppose there is some projection from matrix to vector (in julia this is the `vec` function which gives the matrix in column major order) giving us vectors $n_v$, $x_v$.
$$
n_v(x_v) = n_v
\begin{pmatrix}
x_{1,1} \\
\vdots  \\
x_{3,10}
\end{pmatrix}
= 
\begin{pmatrix}
n_{1,1} \\
\vdots  \\
n_{2,10}
\end{pmatrix}
$$
Now we can compute the jacobian for such "vectorized" function
$$
n_v'(x_v) = W \otimes I_{3},
$$
where $I_{3}$ is identity 3x3 and $\otimes$ is Kronecker/tensor product of matrices. This may look complicated but in reality, when we multiply this from the left with the "vectorized" gradient $\Delta_v$, we get that the adjoint operation boils down to a simple matrix multiplication of
$$
\Delta^TW
$$

It is often the case, that the Jacobi matrix of "vectorized" function does have a nice structure, that allows us to write the adjoints in a more effective way instead of computing the whole Jacobi matrix and doing the multiplication. 

## How to write your own custom adjoints
Though the automatic differentiation in Zygote is really powerfull tool, sometimes you will be forced to write your own adjoint functions. The reasons for this may be 
- performance (functions writte in a way that Zygote understands it may not be most performant solution)
- unsupported operations (array mutation)
- replacing some operations when computing reverse (argmax <-> softmax)
- bragging rights

The users are encouraged to write their own adjoints in order to get the most out of this great tool.

In [78]:
using Zygote: @adjoint

### Simple examples
Let's start again with some simple scalar function of two variables.

In [79]:
add(a, b) = a + b

add (generic function with 1 method)

Adjoints are defined using the `@adjoint` macro, which is a shorthand for defining `pullback(typeof(:add), a, b)`.

In [80]:
@adjoint add(a, b) = add(a, b), Δ -> (Δ, Δ)

The first element of the tuple allows us to redefine function `add`, though here we just call the function itself. The second element is the adjoint function itself, which in this case is really simple, returning the gradient/derivative multiplied by one for each of the inputs.

In [84]:
add_val, add_back = pullback(add, 2, 3)
add_back(2)

(2, 2)

In situation where it may be useful, we can redefine the function itself just for the purposes of AD. This can be usefull in cases, where the code of the adjoint and the function itself is similar and we would like to reuse some of the intermediate results.

In [85]:
@adjoint add(a, b) = begin
    println("I'll be back!?")
    add(a, b) 
end, Δ -> (Δ, Δ)

In [86]:
add_val, add_back = pullback(add, 2, 3)
add_back(1)

I'll be back!?


(1, 1)

Note that the original function defintion is kept untouched 

In [87]:
add(2, 3)

5

Moving to vector functions of multiple variables, recall our function
$$
f(x) = 
\begin{pmatrix}
{x_{1} + x_{2}} \\
{x_{1}x_{2}}
\end{pmatrix}
$$
for which we have defined also Jacobi matrix

In [88]:
df(x_val)

2×2 Array{Float64,2}:
 1.0  1.0
 2.0  1.0

This makes our life easier, asi the only thing we have to do is left multiply that result and return it as a tuple (it always has to be a tuple).

In [90]:
@adjoint f(x) = f(x), Δ -> (Δ'*df(x),)

In [91]:
f_val, f_back = pullback(f, x_val)
f_back([0.0, 1.0])

([2.0 1.0],)

### Gradcheck
How do we check if the adjoint we have defined is correct?
- Zygote itself (only if the automatic pullback works)
- numerical derivatives (using FiniteDifferences)
- by hand

In [92]:
using FiniteDifferences

We use some high order method for computing the 1st derivative and generate an input for the adjoint function.

In [93]:
method = central_fdm(4, 1)

FiniteDifferenceMethod:
  order of method:       4
  order of derivative:   1
  grid:                  [-2, -1, 1, 2]
  coefficients:          [0.08333333333333333, -0.6666666666666666, 0.6666666666666666, -0.08333333333333333]


In [94]:
Δ = rand(2)

2-element Array{Float64,1}:
 0.16439920092321825
 0.6419096254557639

In [95]:
Jf = jacobian(method, f, x_val)[1]

2×2 Array{Float64,2}:
 1.0  1.0
 2.0  1.0

Ussually the results of numerical methods are not this exact, however this is just a toy example.

In [96]:
Δ'*Jf

1×2 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.44822  0.806309

In [97]:
f_back(Δ)[1]

1×2 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.44822  0.806309

In order to compare the output we use the `isapprox` function with some absolute tolerance.

In [98]:
isapprox(Δ'*Jf, f_back(Δ)[1], atol=1e-3)

true

### Caveats with closures
If there are some variables, that the function closes over or if we use a type as a functor (calling the type itself), which is often the case with neural network modules.

In [100]:
addg = let b = 3; a -> a + b end

#47 (generic function with 1 method)

In [101]:
addg_val, addg_back = Zygote._pullback(addg, 4)

(7, ∂(λ))

In [102]:
addg_back(1)

((b = 1,), 1)

The first output of the tuple is a named tuple that contains the gradient with respect to all the variables that the function closes over. Does not appear useful yet, but is a vital part of how to work with paramameters of complex ML models.

In [103]:
struct Linear
    W
    b
end

(l::Linear)(x) = l.W * x .+ l.b

In [ ]:
struct Embedding{S}
  W::S
end

function (a::Embedding)(x)
  W = a.W
  W*x
end

In [104]:
l = Linear(rand(6,10), rand(6))
x = rand(10,5);

In [105]:
l_val, l_back = Zygote._pullback(l, x)

([3.9091815601501514 4.194532038917567 … 3.645981102698167 3.5710408308150914; 3.2499352827275594 3.854240092470749 … 3.292209451334201 2.414990365834626; … ; 3.6702611565633925 4.215870932481884 … 3.419347467983264 3.286336142978225; 3.6616017283372315 4.351396655842046 … 3.5814081629673913 3.2663388518418692], ∂(λ))

In [106]:
Δ = rand(6,5)

6×5 Array{Float64,2}:
 0.572452   0.767901  0.571563  0.745279   0.253134
 0.287731   0.945394  0.938737  0.723804   0.583865
 0.194583   0.37873   0.251414  0.0765259  0.761624
 0.0362843  0.422349  0.614106  0.811824   0.248025
 0.199251   0.443525  0.525148  0.120608   0.298482
 0.775933   0.112187  0.361195  0.314794   0.93278

In [109]:
l_back(Δ)[1]

(W = [1.7631509722695402 1.4148261264120947 … 1.5814256446787824 1.7384500209777871; 1.9957104139648594 1.8605966661599922 … 2.1080481492983725 1.8774622663054994; … ; 0.8743028895548783 0.9443006221467559 … 0.9649184775150219 0.8189675173481034; 1.1904317777711952 1.699724090819081 … 1.1537645074941454 1.1736159424785109], b = [2.9103276961070055, 3.479530742962879, 1.6628763429402746, 2.1325887535448134, 1.5870145953699815, 2.4968902144330825])

You can see that in the first argument is again named tuple indexed by the closed variables, which is then used in cases, where we are looking for gradients with repsect to parameters.

## Pushing the limits
As an experiment let's try to use custom Temperature types defined in the types_dispatch notebook.

In [110]:
include("./temps.jl")

/ (generic function with 188 methods)

Suppose we have a list of measured temeratures on 10 different locations around the city and measurments for one place of interest.

In [111]:
city_temps = Kelvin.(273.15 .+ 20 .* rand(10, 100))
traget_temps = Celsius.(20 .* rand(100))

100-element Array{Celsius,1}:
 14.090868430345193°C
 3.85306390715924°C
 18.489742720823°C
 11.24821942663817°C
 10.26769819412213°C
 9.65190409379042°C
 10.703565315495807°C
 10.762412771899266°C
 12.115684026222336°C
 9.573070895692219°C
 13.076254612275648°C
 5.865732564318908°C
 18.186441341041952°C
 ⋮
 16.054638105156393°C
 12.025644433872094°C
 12.070442346589733°C
 18.773196713128996°C
 10.725609958844004°C
 6.093962371538781°C
 10.040733267570388°C
 17.108614368600907°C
 12.38627136327095°C
 15.5345157337112°C
 2.5413213547434887°C
 13.703772769859368°C

We would like to train a model, that could predict the temperature at the place of interest.

In [112]:
using Flux: Dense, Chain, relu, params, gradient

In [114]:
W1 = rand(5, 10)
b1 = Kelvin.(10 .* rand(5))

5-element Array{Kelvin,1}:
 7.54334820985618K
 5.910512319631525K
 4.086946902211778K
 9.309459954512796K
 5.596412918578335K

In [115]:
W2 = rand(1, 5)
b2 = Kelvin.(10 .* rand(1))

1-element Array{Kelvin,1}:
 1.8574831260744262K

In [116]:
model = Chain(Dense(W1, b1), Dense(W2, b2))

Chain(Dense(10, 5), Dense(5, 1))

In [117]:
model(city_temps)

1×100 Array{Kelvin,2}:
 2227.61K  2236.44K  2237.37K  2248.1K  …  2255.26K  2251.86K  2258.16K

In [118]:
function loss(x,y)
    d = (model(x) .- y)
    (sum(d.*d)/length(y)).value
end

loss (generic function with 1 method)

In [121]:
loss(city_temps, traget_temps)K

3.847373889915592e8K

In [120]:
p = params(model)

Params([[0.8429186955054184 0.8460538219627036 … 0.08405563269499838 0.019530007405989425; 0.42421770799634806 0.6265710056829568 … 0.5237126730845716 0.26961970269466184; … ; 0.4008783022489102 0.35410418888853656 … 0.7173490836988075 0.14009063677286338; 0.21883594260782413 0.6510238018996606 … 0.03359196515218321 0.3710076250917347], [0.47558814373007907 0.7509372915236019 … 0.49183587425809483 0.24210862242121944]])

In [20]:
Zygote.@adjoint Kelvin(t) = Kelvin(t), Δ -> (Δ.value,)

In [122]:
grads = gradient(() -> loss(city_temps, traget_temps), p)

Grads(...)

In [123]:
[grads[pp] for pp in p]

2-element Array{Array{Kelvin,2},1}:
 [5.286899781456846e7K 5.290390608873528e7K … 5.274242594376985e7K 5.278023292046115e7K; 8.347832583263853e7K 8.35334447947073e7K … 8.327847320995726e7K 8.333816912343103e7K; … ; 5.467518504001134e7K 5.471128590116617e7K … 5.454428903776356e7K 5.4583387631114036e7K; 2.6914128113631696e7K 2.6931898939673204e7K … 2.684969391425073e7K 2.686894038867469e7K]
 [5.334685172379714e8K 3.242625302451763e8K … 4.3210930057008064e8K 6.211720579046099e8K]

## More practical examples

## Best practices for using Zygote
- think in Wengert lists!
- try to use as few getindex operations as possible
- avoid complicated for loops
- check the manual adjoints for correctness